In [1]:
import pandas as pd
import glob
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
import numpy as np
import re
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType
from pyspark.sql.types import IntegerType
import matplotlib.pyplot as plt
from pyspark.sql.functions import lit


In [2]:
# Make a new folder for random forest preiction for rental price
# Directory
directory = "random_forest_pred"
  
# Parent Directory path
parent_dir = "../data/curated/"

# Path
path = os.path.join(parent_dir, directory)

# Create the directory
os.mkdir(path)

In [4]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)
parent_dir = "../data/curated/merged_dataset"


sel = RandomForestRegressor(n_estimators = 100)

for filename in os.listdir(parent_dir):
    if filename != "2022_merged_data.csv":
        merged_df_yr = spark.read.csv(parent_dir + "/" + filename, header=True)

        merged_df_yr = merged_df_yr.drop("address","latitude","longitude","postcode","sa2_2016")
        
        for c in merged_df_yr.columns:
            if (c not in  ['address', 'residence_type']):
                merged_df_yr = merged_df_yr.withColumn(c,merged_df_yr[c].cast(FloatType())) 

        merged_df_yr = merged_df_yr.toPandas()

        merged_df_yr['residence_type'] = merged_df_yr['residence_type'].astype('category')
        merged_df_yr['residence_type'] = merged_df_yr['residence_type'].cat.codes

        merged_df_yr.iloc[:, 13:21] = merged_df_yr.iloc[:, 13:21].replace(np.nan, 99999)

        merged_df_yr.rename(columns = {'gdp(USD Millioins)':'gdp', 'saving_rate(% of GDP)':'saving_rate'}, inplace = True)
        
        merged_df_yr = merged_df_yr.dropna()
   
        X = merged_df_yr.drop(['weekly_rent'], axis=1)
        y = np.log(merged_df_yr['weekly_rent'])
        # Train with the whole dataset for the actual prediction for the next 5 years (This training is not for feature enginnering & accuracy test)
        sel.fit(X, y)

accuracy_input = spark.read.csv(parent_dir + "/" + "2022_merged_data.csv", header=True)
accuracy_input = accuracy_input.drop("address","latitude","longitude","postcode","sa2_2016")
        
for c in accuracy_input.columns:
    if (c not in  ['address', 'residence_type']):
        accuracy_input = accuracy_input.withColumn(c,accuracy_input[c].cast(FloatType())) 

accuracy_input = accuracy_input.toPandas()

accuracy_input['residence_type'] = accuracy_input['residence_type'].astype('category')
accuracy_input['residence_type'] = accuracy_input['residence_type'].cat.codes

accuracy_input.iloc[:, 13:21] = accuracy_input.iloc[:, 13:21].replace(np.nan, 99999)

accuracy_input.rename(columns = {'gdp(USD Millioins)':'gdp', 'saving_rate(% of GDP)':'saving_rate'}, inplace = True)
        
accuracy_input = accuracy_input.dropna()
   
X_accuracy = accuracy_input.drop(['weekly_rent'], axis=1)
y_accuracy_true = np.log(accuracy_input['weekly_rent'])

prediction = sel.predict(X_accuracy)

22/10/04 23:18:14 WARN Utils: Your hostname, Hyunjin-Win11 resolves to a loopback address: 127.0.1.1; using 192.168.245.16 instead (on interface eth0)
22/10/04 23:18:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/04 23:18:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Check the accuracy before predicting with the trained random forest regressor
y_accuracy_true = np.exp(y_accuracy_true)
prediction = np.exp(prediction)
mse = mean_squared_error(y_accuracy_true, prediction)
rmse = mse**.5
print(mse)
print(rmse)

# Calculate the absolute errors
errors = np.abs(prediction - y_accuracy_true)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.round(np.mean(errors), 2), 'degrees.')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_accuracy_true)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', np.round(accuracy, 2), '%.')

41158.655941285
202.8759619602209
Mean Absolute Error: 103.06 degrees.
Accuracy: 83.02 %.


In [7]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)
parent_dir = "../data/curated/merged_dataset"

sel_pred = RandomForestRegressor(n_estimators = 100)

for filename in os.listdir(parent_dir):
    print(filename, "----------------------------------------------------------------------------------------------------------------")
    merged_df_yr = spark.read.csv(parent_dir + "/" + filename, header=True)

    merged_df_yr = merged_df_yr.drop("address","latitude","longitude","postcode","sa2_2016")
    
    for c in merged_df_yr.columns:
        if (c not in  ['address', 'residence_type']):
            merged_df_yr = merged_df_yr.withColumn(c,merged_df_yr[c].cast(FloatType())) 

    merged_df_yr = merged_df_yr.toPandas()

    merged_df_yr['residence_type'] = merged_df_yr['residence_type'].astype('category')
    merged_df_yr['residence_type'] = merged_df_yr['residence_type'].cat.codes

    merged_df_yr.iloc[:, 13:21] = merged_df_yr.iloc[:, 13:21].replace(np.nan, 99999)

    merged_df_yr.rename(columns = {'gdp(USD Millioins)':'gdp', 'saving_rate(% of GDP)':'saving_rate'}, inplace = True)
    
    merged_df_yr = merged_df_yr.dropna()

    
    X = merged_df_yr.drop(['weekly_rent'], axis=1)
    y = np.log(merged_df_yr['weekly_rent'])
    # Train with the whole dataset for the actual prediction for the next 5 years (This training is not for feature enginnering & accuracy test)
    sel_pred.fit(X, y)

2013_merged_data.csv ----------------------------------------------------------------------------------------------------------------
2021_merged_data.csv ----------------------------------------------------------------------------------------------------------------
2020_merged_data.csv ----------------------------------------------------------------------------------------------------------------
2022_merged_data.csv ----------------------------------------------------------------------------------------------------------------
2018_merged_data.csv ----------------------------------------------------------------------------------------------------------------
2014_merged_data.csv ----------------------------------------------------------------------------------------------------------------
2017_merged_data.csv ----------------------------------------------------------------------------------------------------------------
2015_merged_data.csv -----------------------------------------

In [22]:
# Predict for the next 5 years
parent_dir = "../data/curated/2023_2027_data"

for filename in os.listdir(parent_dir):
    merged_df_yr = spark.read.csv(parent_dir + "/" + filename, header=True)

    # Extract year from the file name 
    which_year = re.findall(r'\d+', filename)

    # Add year column to the dataset to fit the input into the model
    merged_df_yr = merged_df_yr.withColumn("year", lit(which_year[0]))

    for c in merged_df_yr.columns:
        if c not in ["year", "sa2_2021"]:
            merged_df_yr = merged_df_yr.withColumn(c,merged_df_yr[c].cast(FloatType())) 
        elif c in ["year", "sa2_2021"]:
            merged_df_yr = merged_df_yr.withColumn(c,merged_df_yr[c].cast(IntegerType())) 

    merged_df_yr = merged_df_yr.toPandas()
    print(merged_df_yr["sa2_2021"])
    merged_df_yr.rename(columns = {'gdp(USD Millioins)':'gdp', 'saving_rate(% of GDP)':'saving_rate'}, inplace = True)
    merged_df_yr['residence_type'] = merged_df_yr['residence_type'].astype('category')
    merged_df_yr['residence_type'] = merged_df_yr['residence_type'].cat.codes
    merged_df_yr.dropna(inplace=True)

    # Reorder the columns 
    merged_df_yr_reordered = merged_df_yr[['year', 'sa2_2021', 'residence_type', 'nbed', 'nbath', 'ncar',
       'min_distance_to_cbd', 'min_distance_to_park', 'min_distance_to_prim',
       'min_distance_to_second', 'min_distance_to_train',
       'min_distance_to_hosp', 'min_distance_to_poli', 'min_distance_to_shop',
       'gdp', 'saving_rate', 'income_per_person', 'population_density',
       'crime_cases']]

    # Predict with random forest tree
    prediction = sel_pred.predict(merged_df_yr_reordered)
    prediction = np.exp(prediction)
    new_csv_name = "../data/curated/random_forest_pred/" + filename
    print(merged_df_yr_reordered['sa2_2021'].astype(int))
    data = {'year': merged_df_yr_reordered['year'],
            'sa2_2021': merged_df_yr_reordered['sa2_2021'],
            'predicted_price': prediction }
    df = pd.DataFrame(data)
    print(df)
    df.to_csv(new_csv_name)

0        201011001
1        201011001
2        201011001
3        201011001
4        201011001
           ...    
10409    217041480
10410    217041480
10411    217041480
10412    217041480
10413    217041480
Name: sa2_2021, Length: 10414, dtype: int32
0        201011001
1        201011001
2        201011001
3        201011001
4        201011001
           ...    
10409    217041480
10410    217041480
10411    217041480
10412    217041480
10413    217041480
Name: sa2_2021, Length: 10414, dtype: int64
       year   sa2_2021  predicted_price
0      2023  201011001       169.240076
1      2023  201011001       247.650383
2      2023  201011001       251.456622
3      2023  201011001       177.500942
4      2023  201011001       290.133341
...     ...        ...              ...
10409  2023  217041480       314.601953
10410  2023  217041480       330.345853
10411  2023  217041480       387.327074
10412  2023  217041480       385.634188
10413  2023  217041480       381.143056

[10414 rows x